# Random-to-Random

In [1]:
import random as r
import itertools as it
from typing import *

In [2]:
def get_deck(n):
    return list(range(1, n + 1))

def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)

def choose(n, k):
    return factorial(n) / (factorial(n - k) * factorial(k))

def transform(deck, perm):
    new_deck = deck.copy()
    for cycle in perm:
        temp = new_deck[cycle[0] - 1]
        for i in range(len(cycle)):
            nxt = cycle[(i + 1) % len(cycle)] - 1
            temp, new_deck[nxt] = new_deck[nxt], temp
    return new_deck

In [106]:
class Node:
    def __init__(self, deck:list, marked:Set=set(), depth:int=0, probability:float=1, parent=None):
        self.deck = deck
        self.n = len(deck)
        self.marked = marked
        self.children = []
        self.depth = depth # number of shuffles that have been made
        self.probability = probability # probability of chosing this node from among its siblings
        self.parent = parent
    
    def is_full(self):
        return len(self.marked) == self.n
    
    def __str__(self, indent=0):
        return (" " * (indent * 6)) + ("\n" + (" " * (indent * 6))).join([
            "_" * (self.n * 2 + 1),
            "|" + "|".join([" ", "*"][card in self.marked] for card in self.deck) + "|" + f"   depth: {self.depth}",
            "|" + "|".join(str(card) for card in self.deck) + "|" + f"    prob: {self.probability}",
            "‾" * (self.n * 2 + 1),
        ])
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self, shuffle:str, verbose:bool=False):
        """
        make a copy of the node for each possible shuffle
        put those copies in the children of the current node
        compute and record the transition probabilities for each
        """
        self.children = []
        
        if shuffle == "rtr":
            """
            - Remove a card u.a.r. [n choices]
            - Insert it u.a.r. [n choices]
            """
            for position, card in enumerate(self.deck):
                for new_loc in range(self.n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (self.n * self.n), self)
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering
                    
        if shuffle == "modified_rtr_1":
            """
            - Remove a card u.a.r [n choices]
            - Insert it u.a.r in any position except below it (or on the top if the card removes was at the bottom) [n - 1 choices]
            """
            for position, card in enumerate(self.deck):
                for pre_new_loc in range(position + 2, position + 1 + n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_loc = pre_new_loc % n
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (self.n * (self.n - 1)), self)
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering

        if shuffle == "modified_rtr_2":
            """
            generators
            """
            size_of_S = self.n + (self.n - 2) * (self.n - 1)
            # id
            self.children.append(Node(self.deck.copy(), self.marked.copy(), self.depth + 1, 1 / size_of_S, self))
            # transpositions
            for i in range(self.n - 1):
                new_deck = self.deck.copy()
                new_marked = self.marked.copy()
                if new_deck[i] in new_marked:
                    new_marked.add(new_deck[i + 1])
                elif new_deck[i + 1] in new_marked:
                    new_marked.add(new_deck[i])
#                 else:
#                     new_marked.add(new_deck[i])
                new_deck[i], new_deck[i + 1] = new_deck[i + 1], new_deck[i]
                self.children.append(Node(new_deck, new_marked, self.depth + 1, 1 / size_of_S, self))
            # cycles
            for length in range(3, self.n + 1):
                for initial in range(self.n - length + 1):
                    for direction in [1, -1]:
                        new_deck = self.deck.copy()
                        new_marked = self.marked.copy()
                        a, b = (initial, initial + length - 1)[::direction]
                        card = new_deck[b]
                        new_marked.add(card)
                        new_deck.remove(card)
                        new_deck.insert(a, card)
                        self.children.append(Node(new_deck, new_marked, self.depth + 1, 1 / size_of_S, self))
        
        if shuffle == "thesis":
            """
            - Remove a card u.a.r. [n choices]
            - Insert it u.a.r. [n choices]
            - mark it if it was inserted into a critical position (below any marked card or above first marked card)
            """
            for position, card in enumerate(self.deck):
                for new_loc in range(self.n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    if verbose: print(position, '->', new_loc)
                    if card in self.marked:
                        pass
                    elif len(new_marked) == 0:
                        new_marked.add(card)
                        if verbose: print("first")
                    elif new_loc != 0 and new_deck[new_loc - 1] in self.marked:
                        if verbose: print("below marked")
                        new_marked.add(card)
                    else:
                        first_marked = min([new_deck.index(c) for c in self.marked])
                        if verbose: print("first marked:", first_marked)
                        if first_marked - 1 == new_loc:
                            if verbose: print("above first marked")
                            new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (self.n * self.n), self)
                    self.children.append(node)
                    if verbose: print(node)
                    if verbose: print("-" * 100)
            self.children = self.children[::-1] # to fix annoying ordering
            
            
        if shuffle == "modified_thesis":
            """
            - Remove a card u.a.r. [n choices]
            - Insert it u.a.r. [n choices]
            - mark it if it was inserted below any marked card or (above any marked card with uniform probability)
            """
            for position, card in enumerate(self.deck):
                for new_loc in range(self.n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_deck.insert(new_loc, card)
                    if verbose: print(position, '->', new_loc)
                    if len(self.marked) == 0:
                        if verbose: print("first card")
                        new_marked = self.marked.copy()
                        new_marked.add(card)
                        node = Node(new_deck.copy(), new_marked, self.depth + 1, 1 / (self.n * self.n), self)
                        self.children.append(node)
                        if verbose: print(node)
                        if verbose: print("-" * 100)
                    else:
                        for m_card in self.marked:
                            new_marked = self.marked.copy()
                            if verbose: print("m_card:", m_card)
                            if card in self.marked:
                                if verbose: print("already marked")
                                pass
                            elif new_loc != 0 and new_deck[new_loc - 1] in self.marked:
                                if verbose: print("below marked")
                                new_marked.add(card)
                            elif new_loc + 1 != len(new_deck) and new_deck[new_loc + 1] == m_card:
                                if verbose: print("above m_card")
                                new_marked.add(card)
                            node = Node(new_deck.copy(), new_marked, self.depth + 1, 1 / (self.n * self.n * len(self.marked)), self)
                            self.children.append(node)
                            if verbose: print(node)
                        if verbose: print("-" * 100)
            self.children = self.children[::-1] # to fix annoying ordering

In [117]:
node = Node([1,2,3,4], marked=set([2,3]))
print(node)

_________
| |*|*| |   depth: 0
|1|2|3|4|    prob: 1
‾‾‾‾‾‾‾‾‾


In [118]:
node.generate_children("modified_thesis", verbose=True)

0 -> 0
m_card: 2
above m_card
_________
|*|*|*| |   depth: 1
|1|2|3|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
m_card: 3
_________
| |*|*| |   depth: 1
|1|2|3|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
----------------------------------------------------------------------------------------------------
0 -> 1
m_card: 2
below marked
_________
|*|*|*| |   depth: 1
|2|1|3|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
m_card: 3
below marked
_________
|*|*|*| |   depth: 1
|2|1|3|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
----------------------------------------------------------------------------------------------------
0 -> 2
m_card: 2
below marked
_________
|*|*|*| |   depth: 1
|2|3|1|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
m_card: 3
below marked
_________
|*|*|*| |   depth: 1
|2|3|1|4|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
----------------------------------------------------------------------------------------------------
0 -> 3
m_card: 2
_________
|*|*| | |   depth: 1
|2|3|4|1|    prob: 0.03125
‾‾‾‾‾‾‾‾‾
m_card: 3
_________
|*|*| | |   depth: 1
|2|3|4|1|    prob

In [73]:
class Pair_Node:
    def __init__(self, deck:list, marked:Set=set(), depth:int=0, probability:float=1, parent=None):
        self.deck = deck
        self.n = len(deck)
        self.marked = marked
        self.children = []
        self.depth = depth # number of shuffles that have been made
        self.probability = probability # probability of chosing this node from among its siblings
        self.parent = parent
        
    def is_full(self):
        return len(self.marked) == choose(self.n, 2)
    
    def __str__(self, indent=0):
        return (" " * (indent * 6)) + ("\n" + (" " * (indent * 6))).join([
            "_" * (self.n * 2 + 1) + "   marked: " + str(self.marked),
            "|" + "|".join(str(card) for card in self.deck) + "|" + f"    prob: {self.probability}",
            "‾" * (self.n * 2 + 1) + f"   depth: {self.depth}",
        ]) + "\n\n"
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self, shuffle:str):
        self.children = []

        if shuffle == "modified_gen":
            for a, b in it.combinations(range(1, self.n + 1), 2):
                new_marked = self.marked.copy()
                new_marked.add((a,b))
                if b == a + 1:
                    self.children.append(Pair_Node(transform(self.deck.copy(), [(a, b)]), new_marked.copy(), self.depth + 1, 1 / (2 * choose(self.n, 2)), self))
                    self.children.append(Pair_Node(self.deck.copy(), new_marked.copy(), self.depth + 1, 1 / (2 * choose(self.n, 2)), self))
                else:
                    self.children.append(Pair_Node(transform(self.deck.copy(), [list(range(a, b + 1))]), new_marked.copy(), self.depth + 1, 1 / (2 * choose(self.n, 2)), self))
                    self.children.append(Pair_Node(transform(self.deck.copy(), [list(range(b, a - 1, -1))]), new_marked.copy(), self.depth + 1, 1 / (2 * choose(self.n, 2)), self))

In [74]:
class Tree:
    def __init__(self, n:int, shuffle:str):
        self.n = n
        self.top = Node(get_deck(n))
        self.shuffle = shuffle
    
    
    def __str__(self, max_depth:int=None):
        ret = ""
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if max_depth is not None and node.depth > max_depth:
                continue
            ret += node.__str__(node.depth) + "\n"
            for child in node.children:
                stack.append(child)
        return ret
    
    
    def __repr__(self):
        return str(self)
    
    
    def print(self, max_depth:int=None):
        print(self.__str__(max_depth))
    
    
    def expand(self, max_depth:int):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth >= max_depth:
                continue
            if node.children == []:
                node.generate_children(self.shuffle)
            for child in node.children:
                stack.append(child)
        
        
    def get_leaves(self, depth:int, full:bool=False):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth == depth:
                if not full or node.is_full():
                    yield node
            else:
                for child in node.children:
                    stack.append(child)
        return None
    
    
    def count_full_leaves(self, depth:int):
        """
        counts the occurances of each element of S_n at _depth_ that are full
        """
        self.expand(depth)
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            data[str(node.deck)] += 1
        for deck in data:
            print(deck, " ", data[deck])
        return None
    
    
    def prob_report_full_leaves(self, depth:int, verbose:bool=False):
        """
        for each element s of S_n, find the probability that a full node at _depth_ is s
        return true if all probilities are the same, false otherwise
        """
        self.expand(depth)
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            deck = str(node.deck)
            prod = 1
            while node is not None:
                prod *= node.probability
                node = node.parent
            data[deck] += prod
        total = sum(data.values())
        if verbose: 
            print("probability reaching full:", total)
            print()
            for deck in data:
                print(deck, " ", "{:<23}".format(str(data[deck])), data[deck] / total)
        return len(set(data.values())) == 1
            
    
    def P(self, t:int, sigma:list):
        if t < self.n:
            raise BaseException(f"t must be at least n={self.n}")
        if sorted(sigma) != get_deck(self.n):
            raise BaseException("sigma must be a permutation of n cards")
        
        self.expand(t)
        
        count = 0
        total = 0
        for node in self.get_leaves(t, full=True):
            total += 1
            if node.deck == sigma:
                count += 1
        return count / total

___

In [123]:
tree = Tree(4, "thesis")
tree.prob_report_full_leaves(5, verbose=True)

probability reaching full: 0.10546875

[1, 2, 3, 4]   0.00439453125           0.041666666666666664
[1, 2, 4, 3]   0.00439453125           0.041666666666666664
[1, 3, 2, 4]   0.00439453125           0.041666666666666664
[1, 3, 4, 2]   0.00439453125           0.041666666666666664
[1, 4, 2, 3]   0.00439453125           0.041666666666666664
[1, 4, 3, 2]   0.00439453125           0.041666666666666664
[2, 1, 3, 4]   0.00439453125           0.041666666666666664
[2, 1, 4, 3]   0.00439453125           0.041666666666666664
[2, 3, 1, 4]   0.00439453125           0.041666666666666664
[2, 3, 4, 1]   0.00439453125           0.041666666666666664
[2, 4, 1, 3]   0.00439453125           0.041666666666666664
[2, 4, 3, 1]   0.00439453125           0.041666666666666664
[3, 1, 2, 4]   0.00439453125           0.041666666666666664
[3, 1, 4, 2]   0.00439453125           0.041666666666666664
[3, 2, 1, 4]   0.00439453125           0.041666666666666664
[3, 2, 4, 1]   0.00439453125           0.041666666666666664
[

True

In [123]:
tree.print(max_depth=2)

_________
| | | | |   depth: 0
|1|2|3|4|    prob: 1
‾‾‾‾‾‾‾‾‾
      _________
      | | | |*|   depth: 1
      |1|2|3|4|    prob: 0.0625
      ‾‾‾‾‾‾‾‾‾
            _________
            | | | |*|   depth: 2
            |1|2|3|4|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | | |*| |   depth: 2
            |1|2|4|3|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | |*| | |   depth: 2
            |1|4|2|3|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            |*| | | |   depth: 2
            |4|1|2|3|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | | |*|*|   depth: 2
            |1|2|4|3|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | | | |*|   depth: 2
            |1|2|3|4|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | | | |*|   depth: 2
            |1|3|2|4|    prob: 0.0625
            ‾‾‾‾‾‾‾‾‾
            _________
            | | | |*